<a href="https://colab.research.google.com/github/Praxis-QR/RDWH/blob/main/PG24_3D_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![alt text](https://github.com/Praxis-QR/RDWH/raw/main/images/YantraJaalBanner.png)<br>


<hr>

[Prithwis Mukerjee](http://www.linkedin.com/in/prithwis)<br>

PostgreSQL Tutorial based on https://www.postgresqltutorial.com/

In [1]:
from datetime import datetime
import pytz
print('ॐ श्री सरस्वत्यै नमः',datetime.now(pytz.timezone('Asia/Calcutta')))
!python --version
!lsb_release -a

ॐ श्री सरस्वत्यै नमः 2025-02-27 05:59:34.533211+05:30
Python 3.11.11
No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.4 LTS
Release:	22.04
Codename:	jammy


#Setup

In [2]:
# Install postgresql server

!apt update > /dev/null
!apt install postgresql > /dev/null
#
import psycopg2
import pandas as pd
#!sudo -u postgres psql -V
!psql -V
!service postgresql start



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


psql (PostgreSQL) 14.15 (Ubuntu 14.15-0ubuntu0.22.04.1)
 * Starting PostgreSQL 14 database server
   ...done.


In [3]:
# The install process creates an Ubuntu user 'postgres' AND a Postgres database user 'postgres' both of which are superusers
#!cat /etc/passwd
# Use these users to create a new database user called 'upraxis' and give it a password 'upass'
#!sudo -u postgres createuser --superuser upraxis
!sudo -u postgres createuser upraxis
!sudo -u postgres psql -U postgres -c "ALTER USER upraxis PASSWORD 'upass';"
# Change the database superuser 'postgres' to a non-default value 'pass'
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pass';"
# Use ubuntu superuser to create database dvdrental and set its owner to upraxis
!sudo -u postgres createdb -O upraxis dvdrental
#
# Create file to store the password. Else will be prompted each time
!echo "localhost:5432:dvdrental:postgres:pass" > ~/.pgpass
!echo "localhost:5432:dvdrental:upraxis:upass" >> ~/.pgpass
!chmod 0600 ~/.pgpass
!cat ~/.pgpass

ALTER ROLE
ALTER ROLE
localhost:5432:dvdrental:postgres:pass
localhost:5432:dvdrental:upraxis:upass


## Create, Load Data

In [4]:
!wget -q https://github.com/Praxis-QR/RDWH/raw/main/data/dvdrental.zip
#!wget https://www.postgresqltutorial.com/wp-content/uploads/2019/05/dvdrental.zip
!unzip dvdrental.zip
!ls dvd*.*

Archive:  dvdrental.zip
  inflating: dvdrental.tar           
dvdrental.tar  dvdrental.zip


In [5]:
# Need the Ubuntu user postgres to run the pg_restore utility to recreate database from export
#
!sudo -u postgres pg_restore -d dvdrental /content/dvdrental.tar

In [6]:
# 15 tables created and loaded with data
!psql -h localhost -p 5432 -U postgres -d dvdrental -c '\dt+ public.*'

                                         List of relations
 Schema |     Name      | Type  |  Owner   | Persistence | Access method |    Size    | Description 
--------+---------------+-------+----------+-------------+---------------+------------+-------------
 public | actor         | table | postgres | permanent   | heap          | 40 kB      | 
 public | address       | table | postgres | permanent   | heap          | 88 kB      | 
 public | category      | table | postgres | permanent   | heap          | 8192 bytes | 
 public | city          | table | postgres | permanent   | heap          | 64 kB      | 
 public | country       | table | postgres | permanent   | heap          | 8192 bytes | 
 public | customer      | table | postgres | permanent   | heap          | 96 kB      | 
 public | film          | table | postgres | permanent   | heap          | 464 kB     | 
 public | film_actor    | table | postgres | permanent   | heap          | 264 kB     | 
 public | film_category | t

![alt text](https://raw.githubusercontent.com/Praxis-QR/RDWH/main/images/dvd-rental-sample-database-diagram.png)<br>

In [7]:
!psql -h localhost -p 5432 -U postgres -d dvdrental -c 'select * from actor limit 10'
#the date data should have loaded correctly

 actor_id | first_name |  last_name   |      last_update       
----------+------------+--------------+------------------------
        1 | Penelope   | Guiness      | 2013-05-26 14:47:57.62
        2 | Nick       | Wahlberg     | 2013-05-26 14:47:57.62
        3 | Ed         | Chase        | 2013-05-26 14:47:57.62
        4 | Jennifer   | Davis        | 2013-05-26 14:47:57.62
        5 | Johnny     | Lollobrigida | 2013-05-26 14:47:57.62
        6 | Bette      | Nicholson    | 2013-05-26 14:47:57.62
        7 | Grace      | Mostel       | 2013-05-26 14:47:57.62
        8 | Matthew    | Johansson    | 2013-05-26 14:47:57.62
        9 | Joe        | Swank        | 2013-05-26 14:47:57.62
       10 | Christian  | Gable        | 2013-05-26 14:47:57.62
(10 rows)



In [8]:
# check users and their authority
!psql -h localhost -p 5432 -U postgres -d dvdrental -c '\du'

                                   List of roles
 Role name |                         Attributes                         | Member of 
-----------+------------------------------------------------------------+-----------
 postgres  | Superuser, Create role, Create DB, Replication, Bypass RLS | {}
 upraxis   |                                                            | {}



In [9]:
# Grant access to user upraxis
!psql -h localhost -p 5432 -U postgres -d dvdrental -c 'GRANT INSERT, UPDATE, SELECT, DELETE ON ALL TABLES IN SCHEMA public TO upraxis;'

GRANT


In [10]:
!psql -h localhost -p 5432 -U upraxis -d dvdrental -c 'select * from actor limit 10'
#the date data should have loaded correctly

 actor_id | first_name |  last_name   |      last_update       
----------+------------+--------------+------------------------
        1 | Penelope   | Guiness      | 2013-05-26 14:47:57.62
        2 | Nick       | Wahlberg     | 2013-05-26 14:47:57.62
        3 | Ed         | Chase        | 2013-05-26 14:47:57.62
        4 | Jennifer   | Davis        | 2013-05-26 14:47:57.62
        5 | Johnny     | Lollobrigida | 2013-05-26 14:47:57.62
        6 | Bette      | Nicholson    | 2013-05-26 14:47:57.62
        7 | Grace      | Mostel       | 2013-05-26 14:47:57.62
        8 | Matthew    | Johansson    | 2013-05-26 14:47:57.62
        9 | Joe        | Swank        | 2013-05-26 14:47:57.62
       10 | Christian  | Gable        | 2013-05-26 14:47:57.62
(10 rows)



## Python Access
https://www.tutorialspoint.com/python_data_access/python_postgresql_database_connection.htm

In [11]:
def runSelect(s):
    #Set up connection

    conn = psycopg2.connect(database="dvdrental", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Setting auto commit true
    conn.autocommit = True
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    #Retrieving data
    cursor.execute(s)
    #Fetching all rows from the table
    result = cursor.fetchall();
    #print(len(result))
    if (len(result)) > 0:                                       # result contains at least one row
        colnames = [desc[0] for desc in cursor.description]     # get column names
        #print(colnames)
        resultdf = pd.DataFrame(result)                         # convert list to dataframe
        resultdf.columns = colnames                             # add column names
        #return resultdf
        return resultdf.style.hide()                            # hide the index
    else:
        print('no records found')
        return

def runCmd(s):
    #Create Connection
    conn = psycopg2.connect(database="dvdrental", user='upraxis', password='upass', host='127.0.0.1', port= '5432')
    #Creating a cursor object using the cursor() method
    cursor = conn.cursor()
    try:
        cursor.execute(s)
        print('Command executed')
        conn.commit()
    except:
        print('Command failed')
    #Closing the connection
    conn.close()

In [12]:
runSelect('select count(*) from film')

count
1000


#ChronoTYM <br>
In Indic phraseology, tantra is knowledge, a yantra is a device to explore this knowledge and a mantra is the code that unlocks both the tantra and its yantra. Chrono/TYM a sci-fi trilogy that recreates this framework where the first novel Chronotantra defines the contours of a tech driven, futuristic society, the second, Chronoyantra, describes a device used to explore the genesis of this society, while the third, Chronomantra finally reveals the code, the key, that unlocks this puzzle.  [More information](http://bit.ly/chronobooks) <br>
![alt text](https://blogger.googleusercontent.com/img/b/R29vZ2xl/AVvXsEjdu_ajYwBRzk5iFYlcs59ZoRMMg6rhNm4JERKspvFJoveJsnfwRlzMICiC37UOja_Bd4sbfLA1pyyBbe0mj3tUfDBXoqCzNUDw3Wux7acoJBaLkWjAvJXfa7drRY2GQRAlFxEY9O7a7raY4RKy7cA3Uxgf5QsaZQ99C9GfXxFSex_rke8ROhAn41t8zOqK/s1640/tymBanner01_3.png)

In [13]:
from datetime import datetime
import pytz
print('tested',datetime.now(pytz.timezone('Asia/Calcutta')))

tested 2025-02-27 06:00:20.535522+05:30
